### Importing Libraries

In [1]:
import re
import nltk
import string
import pickle
import imblearn
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from sklearn.linear_model import Ridge
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score

### Reading Data

In [2]:
df1= pd.read_csv("data.csv",low_memory=False)

In [3]:
df1

,OID_,OWNER,MAIL_NAME,MAIL_ADDRE,MAIL_AD_01,MAIL_AD_02,M_RECPNT,M_USPSBOX,M_ADDRESSN,M_STREETPR,...,M_STREETPO,M_STREETPD,M_SUBOCC,M_PLACENM,M_STATENM,M_ZIPCODE,M_COUNTRY,LEGAL_DESC,Shape_Length,Shape_Area
0,1,AGUILLON HECTOR ETUX NEYDA TOBAR,,409 W AVENUE A,,BELTON TX 76513,,,409,W AVENUE,...,,,,BELTON,TX,76513,,"BELTON ORIGINAL, BLOCK 110, LOT PT I, (75' X 1...",567.120628,15652.887910
1,2,CITY OF BELTON,,PO BOX 120,,BELTON TX 76513,,PO BOX 120,,,...,,,,BELTON,TX,76513,,"BELTON ORIGINAL, BLOCK 110, LOT PT A, (E PT OF...",1364.987519,58986.806740
2,3,BELTON TEXAS REAL ESTATE HOLDINGS LLC,,600 SAGE BRUSH,,BELTON TX 76513,,,600,,...,,,,BELTON,TX,76513,,"PENDLETON, BLOCK 003, LOT PT 2, (S 1/2 OF W 1/...",411.048085,10105.525380
3,4,LEWIS NELDA MAE,,PO BOX 1082,,BELTON TX 76513,,PO BOX 1082,,,...,,,,BELTON,TX,76513,,"BELTON ORIGINAL, BLOCK 107, LOT PT 1, (W 113.4...",989.135441,44147.277460
4,5,MARQUEZ FERNANDO,,PO BOX 983,,BELTON TX 76513,,PO BOX 983,,,...,,,,BELTON,TX,76513,,"RICHARD 2ND, BLOCK 001, LOT 0014, MH LABEL NOT...",838.542479,43946.812030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,1048571,WHITE BARBARA G,,9813 CLOCKTOWER CT,,PLANO TX 75025-6584,,,9813,,...,CT,,,PLANO,TX,75025-6584,,"PASQUINELLIS CASTLEBROOK AT RIDGEVIEW (CPL), B...",325.121316,5037.827226
1048571,1048572,BAUM RAQUEL,,9816 CLOCKTOWER CT,,PLANO TX 75025-6583,,,9816,,...,CT,,,PLANO,TX,75025-6583,,"PASQUINELLIS CASTLEBROOK AT RIDGEVIEW (CPL), B...",329.108905,5123.142305
1048572,1048573,AZIZ HOOR,,9817 CLOCKTOWER CT,,PLANO TX 75025-6584,,,9817,,...,CT,,,PLANO,TX,75025-6584,,"PASQUINELLIS CASTLEBROOK AT RIDGEVIEW (CPL), B...",315.632426,4294.791267
1048573,1048574,FU QINGRU & PAI LIU &,,6409 LAMBEAU CT,,MCKINNEY TX 75070-9560,,,6409,,...,CT,,,MCKINNEY,TX,75070-9560,,"PASQUINELLIS CASTLEBROOK AT RIDGEVIEW (CPL), B...",315.609891,4294.425418


## Shape of data

In [4]:
df1.shape

(1048575, 21)

## Removing Null Values 

In [5]:
df1=df1.dropna( axis=0, how="any", thresh=None, subset=None, inplace=False)

## Checking Null Values

In [6]:
df1.isnull().sum()

OID_            0
OWNER           0
MAIL_NAME       0
MAIL_ADDRE      0
MAIL_AD_01      0
MAIL_AD_02      0
M_RECPNT        0
M_USPSBOX       0
M_ADDRESSN      0
M_STREETPR      0
M_STREETNM      0
M_STREETPO      0
M_STREETPD      0
M_SUBOCC        0
M_PLACENM       0
M_STATENM       0
M_ZIPCODE       0
M_COUNTRY       0
LEGAL_DESC      0
Shape_Length    0
Shape_Area      0
dtype: int64

In [7]:
df1.head()

,OID_,OWNER,MAIL_NAME,MAIL_ADDRE,MAIL_AD_01,MAIL_AD_02,M_RECPNT,M_USPSBOX,M_ADDRESSN,M_STREETPR,...,M_STREETPO,M_STREETPD,M_SUBOCC,M_PLACENM,M_STATENM,M_ZIPCODE,M_COUNTRY,LEGAL_DESC,Shape_Length,Shape_Area
0,1,AGUILLON HECTOR ETUX NEYDA TOBAR,,409 W AVENUE A,,BELTON TX 76513,,,409,W AVENUE,...,,,,BELTON,TX,76513,,"BELTON ORIGINAL, BLOCK 110, LOT PT I, (75' X 1...",567.120628,15652.88791
1,2,CITY OF BELTON,,PO BOX 120,,BELTON TX 76513,,PO BOX 120,,,...,,,,BELTON,TX,76513,,"BELTON ORIGINAL, BLOCK 110, LOT PT A, (E PT OF...",1364.987519,58986.80674
2,3,BELTON TEXAS REAL ESTATE HOLDINGS LLC,,600 SAGE BRUSH,,BELTON TX 76513,,,600,,...,,,,BELTON,TX,76513,,"PENDLETON, BLOCK 003, LOT PT 2, (S 1/2 OF W 1/...",411.048085,10105.52538
3,4,LEWIS NELDA MAE,,PO BOX 1082,,BELTON TX 76513,,PO BOX 1082,,,...,,,,BELTON,TX,76513,,"BELTON ORIGINAL, BLOCK 107, LOT PT 1, (W 113.4...",989.135441,44147.27746
4,5,MARQUEZ FERNANDO,,PO BOX 983,,BELTON TX 76513,,PO BOX 983,,,...,,,,BELTON,TX,76513,,"RICHARD 2ND, BLOCK 001, LOT 0014, MH LABEL NOT...",838.542479,43946.81203


In [8]:
df1.tail()

,OID_,OWNER,MAIL_NAME,MAIL_ADDRE,MAIL_AD_01,MAIL_AD_02,M_RECPNT,M_USPSBOX,M_ADDRESSN,M_STREETPR,...,M_STREETPO,M_STREETPD,M_SUBOCC,M_PLACENM,M_STATENM,M_ZIPCODE,M_COUNTRY,LEGAL_DESC,Shape_Length,Shape_Area
1048570,1048571,WHITE BARBARA G,,9813 CLOCKTOWER CT,,PLANO TX 75025-6584,,,9813,,...,CT,,,PLANO,TX,75025-6584,,"PASQUINELLIS CASTLEBROOK AT RIDGEVIEW (CPL), B...",325.121316,5037.827226
1048571,1048572,BAUM RAQUEL,,9816 CLOCKTOWER CT,,PLANO TX 75025-6583,,,9816,,...,CT,,,PLANO,TX,75025-6583,,"PASQUINELLIS CASTLEBROOK AT RIDGEVIEW (CPL), B...",329.108905,5123.142305
1048572,1048573,AZIZ HOOR,,9817 CLOCKTOWER CT,,PLANO TX 75025-6584,,,9817,,...,CT,,,PLANO,TX,75025-6584,,"PASQUINELLIS CASTLEBROOK AT RIDGEVIEW (CPL), B...",315.632426,4294.791267
1048573,1048574,FU QINGRU & PAI LIU &,,6409 LAMBEAU CT,,MCKINNEY TX 75070-9560,,,6409,,...,CT,,,MCKINNEY,TX,75070-9560,,"PASQUINELLIS CASTLEBROOK AT RIDGEVIEW (CPL), B...",315.609891,4294.425418
1048574,1048575,HENDRICKS MADELYN W &,,9821 CLOCKTOWER CT,,PLANO TX 75025-6584,,,9821,,...,CT,,,PLANO,TX,75025-6584,,"PASQUINELLIS CASTLEBROOK AT RIDGEVIEW (CPL), B...",315.631703,4294.775320


### Cleaning data

In [9]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [10]:
df1["Pre-Process"] = df1["OWNER"].apply(wordopt)

C:\Users\Abu Huraira\AppData\Local\Temp\ipykernel_6456\654537613.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["Pre-Process"] = df1["OWNER"].apply(wordopt)


In [11]:
df1.head()

,OID_,OWNER,MAIL_NAME,MAIL_ADDRE,MAIL_AD_01,MAIL_AD_02,M_RECPNT,M_USPSBOX,M_ADDRESSN,M_STREETPR,...,M_STREETPD,M_SUBOCC,M_PLACENM,M_STATENM,M_ZIPCODE,M_COUNTRY,LEGAL_DESC,Shape_Length,Shape_Area,Pre-Process
0,1,AGUILLON HECTOR ETUX NEYDA TOBAR,,409 W AVENUE A,,BELTON TX 76513,,,409,W AVENUE,...,,,BELTON,TX,76513,,"BELTON ORIGINAL, BLOCK 110, LOT PT I, (75' X 1...",567.120628,15652.88791,aguillon hector etux neyda tobar
1,2,CITY OF BELTON,,PO BOX 120,,BELTON TX 76513,,PO BOX 120,,,...,,,BELTON,TX,76513,,"BELTON ORIGINAL, BLOCK 110, LOT PT A, (E PT OF...",1364.987519,58986.80674,city of belton
2,3,BELTON TEXAS REAL ESTATE HOLDINGS LLC,,600 SAGE BRUSH,,BELTON TX 76513,,,600,,...,,,BELTON,TX,76513,,"PENDLETON, BLOCK 003, LOT PT 2, (S 1/2 OF W 1/...",411.048085,10105.52538,belton texas real estate holdings llc
3,4,LEWIS NELDA MAE,,PO BOX 1082,,BELTON TX 76513,,PO BOX 1082,,,...,,,BELTON,TX,76513,,"BELTON ORIGINAL, BLOCK 107, LOT PT 1, (W 113.4...",989.135441,44147.27746,lewis nelda mae
4,5,MARQUEZ FERNANDO,,PO BOX 983,,BELTON TX 76513,,PO BOX 983,,,...,,,BELTON,TX,76513,,"RICHARD 2ND, BLOCK 001, LOT 0014, MH LABEL NOT...",838.542479,43946.81203,marquez fernando


## Tokenization

In [12]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
df1["Tokenized"] = df1["Pre-Process"].apply(tokenizer.tokenize)

C:\Users\Abu Huraira\AppData\Local\Temp\ipykernel_6456\3363224063.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["Tokenized"] = df1["Pre-Process"].apply(tokenizer.tokenize)


In [13]:
df1.head()

,OID_,OWNER,MAIL_NAME,MAIL_ADDRE,MAIL_AD_01,MAIL_AD_02,M_RECPNT,M_USPSBOX,M_ADDRESSN,M_STREETPR,...,M_SUBOCC,M_PLACENM,M_STATENM,M_ZIPCODE,M_COUNTRY,LEGAL_DESC,Shape_Length,Shape_Area,Pre-Process,Tokenized
0,1,AGUILLON HECTOR ETUX NEYDA TOBAR,,409 W AVENUE A,,BELTON TX 76513,,,409,W AVENUE,...,,BELTON,TX,76513,,"BELTON ORIGINAL, BLOCK 110, LOT PT I, (75' X 1...",567.120628,15652.88791,aguillon hector etux neyda tobar,"[aguillon, hector, etux, neyda, tobar]"
1,2,CITY OF BELTON,,PO BOX 120,,BELTON TX 76513,,PO BOX 120,,,...,,BELTON,TX,76513,,"BELTON ORIGINAL, BLOCK 110, LOT PT A, (E PT OF...",1364.987519,58986.80674,city of belton,"[city, of, belton]"
2,3,BELTON TEXAS REAL ESTATE HOLDINGS LLC,,600 SAGE BRUSH,,BELTON TX 76513,,,600,,...,,BELTON,TX,76513,,"PENDLETON, BLOCK 003, LOT PT 2, (S 1/2 OF W 1/...",411.048085,10105.52538,belton texas real estate holdings llc,"[belton, texas, real, estate, holdings, llc]"
3,4,LEWIS NELDA MAE,,PO BOX 1082,,BELTON TX 76513,,PO BOX 1082,,,...,,BELTON,TX,76513,,"BELTON ORIGINAL, BLOCK 107, LOT PT 1, (W 113.4...",989.135441,44147.27746,lewis nelda mae,"[lewis, nelda, mae]"
4,5,MARQUEZ FERNANDO,,PO BOX 983,,BELTON TX 76513,,PO BOX 983,,,...,,BELTON,TX,76513,,"RICHARD 2ND, BLOCK 001, LOT 0014, MH LABEL NOT...",838.542479,43946.81203,marquez fernando,"[marquez, fernando]"


## Removing stop words

In [14]:
stopwords=nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    text=[word for word in text if word not in stopwords]
    return text

In [15]:
df1['SW_Rem']=df1['Tokenized'].apply(lambda x:remove_stopwords(x))
df1.head()

C:\Users\Abu Huraira\AppData\Local\Temp\ipykernel_6456\2032337043.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['SW_Rem']=df1['Tokenized'].apply(lambda x:remove_stopwords(x))


,OID_,OWNER,MAIL_NAME,MAIL_ADDRE,MAIL_AD_01,MAIL_AD_02,M_RECPNT,M_USPSBOX,M_ADDRESSN,M_STREETPR,...,M_PLACENM,M_STATENM,M_ZIPCODE,M_COUNTRY,LEGAL_DESC,Shape_Length,Shape_Area,Pre-Process,Tokenized,SW_Rem
0,1,AGUILLON HECTOR ETUX NEYDA TOBAR,,409 W AVENUE A,,BELTON TX 76513,,,409,W AVENUE,...,BELTON,TX,76513,,"BELTON ORIGINAL, BLOCK 110, LOT PT I, (75' X 1...",567.120628,15652.88791,aguillon hector etux neyda tobar,"[aguillon, hector, etux, neyda, tobar]","[aguillon, hector, etux, neyda, tobar]"
1,2,CITY OF BELTON,,PO BOX 120,,BELTON TX 76513,,PO BOX 120,,,...,BELTON,TX,76513,,"BELTON ORIGINAL, BLOCK 110, LOT PT A, (E PT OF...",1364.987519,58986.80674,city of belton,"[city, of, belton]","[city, belton]"
2,3,BELTON TEXAS REAL ESTATE HOLDINGS LLC,,600 SAGE BRUSH,,BELTON TX 76513,,,600,,...,BELTON,TX,76513,,"PENDLETON, BLOCK 003, LOT PT 2, (S 1/2 OF W 1/...",411.048085,10105.52538,belton texas real estate holdings llc,"[belton, texas, real, estate, holdings, llc]","[belton, texas, real, estate, holdings, llc]"
3,4,LEWIS NELDA MAE,,PO BOX 1082,,BELTON TX 76513,,PO BOX 1082,,,...,BELTON,TX,76513,,"BELTON ORIGINAL, BLOCK 107, LOT PT 1, (W 113.4...",989.135441,44147.27746,lewis nelda mae,"[lewis, nelda, mae]","[lewis, nelda, mae]"
4,5,MARQUEZ FERNANDO,,PO BOX 983,,BELTON TX 76513,,PO BOX 983,,,...,BELTON,TX,76513,,"RICHARD 2ND, BLOCK 001, LOT 0014, MH LABEL NOT...",838.542479,43946.81203,marquez fernando,"[marquez, fernando]","[marquez, fernando]"


## Lemmatization

In [16]:
wn= nltk.WordNetLemmatizer()
def lemmatization(text):
    text=[wn.lemmatize(word) for word in text]
    return text

In [17]:
df1['Lemmas']=df1['SW_Rem'].apply(lambda x:lemmatization(x))
df1.head()

C:\Users\Abu Huraira\AppData\Local\Temp\ipykernel_6456\3082508211.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Lemmas']=df1['SW_Rem'].apply(lambda x:lemmatization(x))


,OID_,OWNER,MAIL_NAME,MAIL_ADDRE,MAIL_AD_01,MAIL_AD_02,M_RECPNT,M_USPSBOX,M_ADDRESSN,M_STREETPR,...,M_STATENM,M_ZIPCODE,M_COUNTRY,LEGAL_DESC,Shape_Length,Shape_Area,Pre-Process,Tokenized,SW_Rem,Lemmas
0,1,AGUILLON HECTOR ETUX NEYDA TOBAR,,409 W AVENUE A,,BELTON TX 76513,,,409,W AVENUE,...,TX,76513,,"BELTON ORIGINAL, BLOCK 110, LOT PT I, (75' X 1...",567.120628,15652.88791,aguillon hector etux neyda tobar,"[aguillon, hector, etux, neyda, tobar]","[aguillon, hector, etux, neyda, tobar]","[aguillon, hector, etux, neyda, tobar]"
1,2,CITY OF BELTON,,PO BOX 120,,BELTON TX 76513,,PO BOX 120,,,...,TX,76513,,"BELTON ORIGINAL, BLOCK 110, LOT PT A, (E PT OF...",1364.987519,58986.80674,city of belton,"[city, of, belton]","[city, belton]","[city, belton]"
2,3,BELTON TEXAS REAL ESTATE HOLDINGS LLC,,600 SAGE BRUSH,,BELTON TX 76513,,,600,,...,TX,76513,,"PENDLETON, BLOCK 003, LOT PT 2, (S 1/2 OF W 1/...",411.048085,10105.52538,belton texas real estate holdings llc,"[belton, texas, real, estate, holdings, llc]","[belton, texas, real, estate, holdings, llc]","[belton, texas, real, estate, holding, llc]"
3,4,LEWIS NELDA MAE,,PO BOX 1082,,BELTON TX 76513,,PO BOX 1082,,,...,TX,76513,,"BELTON ORIGINAL, BLOCK 107, LOT PT 1, (W 113.4...",989.135441,44147.27746,lewis nelda mae,"[lewis, nelda, mae]","[lewis, nelda, mae]","[lewis, nelda, mae]"
4,5,MARQUEZ FERNANDO,,PO BOX 983,,BELTON TX 76513,,PO BOX 983,,,...,TX,76513,,"RICHARD 2ND, BLOCK 001, LOT 0014, MH LABEL NOT...",838.542479,43946.81203,marquez fernando,"[marquez, fernando]","[marquez, fernando]","[marquez, fernando]"


## Features Extraction

# Count Vectorization 

In [18]:
vectorizer = CountVectorizer(analyzer='word')
X1 = vectorizer.fit_transform(df1['Lemmas'].astype('str'))
vectorizer.get_feature_names()

C:\Users\Abu Huraira\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['aa',
 'aaa',
 'aaag',
 'aabbha',
 'aabhas',
 'aabvc',
 'aack',
 'aackerlund',
 'aadesh',
 'aadhyashivansh',
 'aadi',
 'aadil',
 'aaditya',
 'aadland',
 'aadrien',
 'aae',
 'aafl',
 'aafreen',
 'aag',
 'aagaard',
 'aahraf',
 'aaj',
 'aak',
 'aakahkumar',
 'aakanksha',
 'aakash',
 'aaker',
 'aakhus',
 'aakrati',
 'aaku',
 'aala',
 'aalaee',
 'aalay',
 'aalderink',
 'aalfs',
 'aalhasan',
 'aalid',
 'aalinejad',
 'aaliya',
 'aaliyah',
 'aalla',
 'aalok',
 'aalseth',
 'aalsma',
 'aalsudani',
 'aam',
 'aamber',
 'aamely',
 'aamer',
 'aamha',
 'aamil',
 'aamina',
 'aamir',
 'aamnon',
 'aamodt',
 'aams',
 'aamshu',
 'aana',
 'aanaadi',
 'aanand',
 'aanandha',
 'aanchal',
 'aanderud',
 'aanenson',
 'aangarai',
 'aangdembe',
 'aanm',
 'aanonsen',
 'aantyah',
 'aanu',
 'aanuoluwa',
 'aaonms',
 'aara',
 'aarabhi',
 'aaradhya',
 'aaraguz',
 'aaran',
 'aarant',
 'aarathi',
 'aarati',
 'aardahl',
 'aardal',
 'aardema',
 'aarden',
 'aardon',
 'aaren',
 'aarey',
 'aarhus',
 'aarica',
 'aarif',
 'aari

In [19]:
model=pickle.load(open('Trained_model.pkl', 'rb'))

In [20]:
OwnerType =model.predict(X1)
df1["OwnerType"] = np.round(OwnerType,0)


C:\Users\Abu Huraira\AppData\Local\Temp\ipykernel_6456\1195921449.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["OwnerType"] = np.round(OwnerType,0)


In [21]:
df1

,OID_,OWNER,MAIL_NAME,MAIL_ADDRE,MAIL_AD_01,MAIL_AD_02,M_RECPNT,M_USPSBOX,M_ADDRESSN,M_STREETPR,...,M_ZIPCODE,M_COUNTRY,LEGAL_DESC,Shape_Length,Shape_Area,Pre-Process,Tokenized,SW_Rem,Lemmas,OwnerType
0,1,AGUILLON HECTOR ETUX NEYDA TOBAR,,409 W AVENUE A,,BELTON TX 76513,,,409,W AVENUE,...,76513,,"BELTON ORIGINAL, BLOCK 110, LOT PT I, (75' X 1...",567.120628,15652.887910,aguillon hector etux neyda tobar,"[aguillon, hector, etux, neyda, tobar]","[aguillon, hector, etux, neyda, tobar]","[aguillon, hector, etux, neyda, tobar]",3
1,2,CITY OF BELTON,,PO BOX 120,,BELTON TX 76513,,PO BOX 120,,,...,76513,,"BELTON ORIGINAL, BLOCK 110, LOT PT A, (E PT OF...",1364.987519,58986.806740,city of belton,"[city, of, belton]","[city, belton]","[city, belton]",2
2,3,BELTON TEXAS REAL ESTATE HOLDINGS LLC,,600 SAGE BRUSH,,BELTON TX 76513,,,600,,...,76513,,"PENDLETON, BLOCK 003, LOT PT 2, (S 1/2 OF W 1/...",411.048085,10105.525380,belton texas real estate holdings llc,"[belton, texas, real, estate, holdings, llc]","[belton, texas, real, estate, holdings, llc]","[belton, texas, real, estate, holding, llc]",0
3,4,LEWIS NELDA MAE,,PO BOX 1082,,BELTON TX 76513,,PO BOX 1082,,,...,76513,,"BELTON ORIGINAL, BLOCK 107, LOT PT 1, (W 113.4...",989.135441,44147.277460,lewis nelda mae,"[lewis, nelda, mae]","[lewis, nelda, mae]","[lewis, nelda, mae]",3
4,5,MARQUEZ FERNANDO,,PO BOX 983,,BELTON TX 76513,,PO BOX 983,,,...,76513,,"RICHARD 2ND, BLOCK 001, LOT 0014, MH LABEL NOT...",838.542479,43946.812030,marquez fernando,"[marquez, fernando]","[marquez, fernando]","[marquez, fernando]",3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,1048571,WHITE BARBARA G,,9813 CLOCKTOWER CT,,PLANO TX 75025-6584,,,9813,,...,75025-6584,,"PASQUINELLIS CASTLEBROOK AT RIDGEVIEW (CPL), B...",325.121316,5037.827226,white barbara g,"[white, barbara, g]","[white, barbara, g]","[white, barbara, g]",4
1048571,1048572,BAUM RAQUEL,,9816 CLOCKTOWER CT,,PLANO TX 75025-6583,,,9816,,...,75025-6583,,"PASQUINELLIS CASTLEBROOK AT RIDGEVIEW (CPL), B...",329.108905,5123.142305,baum raquel,"[baum, raquel]","[baum, raquel]","[baum, raquel]",4
1048572,1048573,AZIZ HOOR,,9817 CLOCKTOWER CT,,PLANO TX 75025-6584,,,9817,,...,75025-6584,,"PASQUINELLIS CASTLEBROOK AT RIDGEVIEW (CPL), B...",315.632426,4294.791267,aziz hoor,"[aziz, hoor]","[aziz, hoor]","[aziz, hoor]",4
1048573,1048574,FU QINGRU & PAI LIU &,,6409 LAMBEAU CT,,MCKINNEY TX 75070-9560,,,6409,,...,75070-9560,,"PASQUINELLIS CASTLEBROOK AT RIDGEVIEW (CPL), B...",315.609891,4294.425418,fu qingru pai liu,"[fu, qingru, pai, liu]","[fu, qingru, pai, liu]","[fu, qingru, pai, liu]",4


In [22]:
df1['OwnerType'] = df1['OwnerType'].map({0: 'Company', 1: 'Farm',2:'NonProfit',3:'Person',4:'Ranch',5:'Trust'})


C:\Users\Abu Huraira\AppData\Local\Temp\ipykernel_6456\3370065657.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['OwnerType'] = df1['OwnerType'].map({0: 'Company', 1: 'Farm',2:'NonProfit',3:'Person',4:'Ranch',5:'Trust'})


In [23]:
df1

,OID_,OWNER,MAIL_NAME,MAIL_ADDRE,MAIL_AD_01,MAIL_AD_02,M_RECPNT,M_USPSBOX,M_ADDRESSN,M_STREETPR,...,M_ZIPCODE,M_COUNTRY,LEGAL_DESC,Shape_Length,Shape_Area,Pre-Process,Tokenized,SW_Rem,Lemmas,OwnerType
0,1,AGUILLON HECTOR ETUX NEYDA TOBAR,,409 W AVENUE A,,BELTON TX 76513,,,409,W AVENUE,...,76513,,"BELTON ORIGINAL, BLOCK 110, LOT PT I, (75' X 1...",567.120628,15652.887910,aguillon hector etux neyda tobar,"[aguillon, hector, etux, neyda, tobar]","[aguillon, hector, etux, neyda, tobar]","[aguillon, hector, etux, neyda, tobar]",Person
1,2,CITY OF BELTON,,PO BOX 120,,BELTON TX 76513,,PO BOX 120,,,...,76513,,"BELTON ORIGINAL, BLOCK 110, LOT PT A, (E PT OF...",1364.987519,58986.806740,city of belton,"[city, of, belton]","[city, belton]","[city, belton]",NonProfit
2,3,BELTON TEXAS REAL ESTATE HOLDINGS LLC,,600 SAGE BRUSH,,BELTON TX 76513,,,600,,...,76513,,"PENDLETON, BLOCK 003, LOT PT 2, (S 1/2 OF W 1/...",411.048085,10105.525380,belton texas real estate holdings llc,"[belton, texas, real, estate, holdings, llc]","[belton, texas, real, estate, holdings, llc]","[belton, texas, real, estate, holding, llc]",Company
3,4,LEWIS NELDA MAE,,PO BOX 1082,,BELTON TX 76513,,PO BOX 1082,,,...,76513,,"BELTON ORIGINAL, BLOCK 107, LOT PT 1, (W 113.4...",989.135441,44147.277460,lewis nelda mae,"[lewis, nelda, mae]","[lewis, nelda, mae]","[lewis, nelda, mae]",Person
4,5,MARQUEZ FERNANDO,,PO BOX 983,,BELTON TX 76513,,PO BOX 983,,,...,76513,,"RICHARD 2ND, BLOCK 001, LOT 0014, MH LABEL NOT...",838.542479,43946.812030,marquez fernando,"[marquez, fernando]","[marquez, fernando]","[marquez, fernando]",Person
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,1048571,WHITE BARBARA G,,9813 CLOCKTOWER CT,,PLANO TX 75025-6584,,,9813,,...,75025-6584,,"PASQUINELLIS CASTLEBROOK AT RIDGEVIEW (CPL), B...",325.121316,5037.827226,white barbara g,"[white, barbara, g]","[white, barbara, g]","[white, barbara, g]",Ranch
1048571,1048572,BAUM RAQUEL,,9816 CLOCKTOWER CT,,PLANO TX 75025-6583,,,9816,,...,75025-6583,,"PASQUINELLIS CASTLEBROOK AT RIDGEVIEW (CPL), B...",329.108905,5123.142305,baum raquel,"[baum, raquel]","[baum, raquel]","[baum, raquel]",Ranch
1048572,1048573,AZIZ HOOR,,9817 CLOCKTOWER CT,,PLANO TX 75025-6584,,,9817,,...,75025-6584,,"PASQUINELLIS CASTLEBROOK AT RIDGEVIEW (CPL), B...",315.632426,4294.791267,aziz hoor,"[aziz, hoor]","[aziz, hoor]","[aziz, hoor]",Ranch
1048573,1048574,FU QINGRU & PAI LIU &,,6409 LAMBEAU CT,,MCKINNEY TX 75070-9560,,,6409,,...,75070-9560,,"PASQUINELLIS CASTLEBROOK AT RIDGEVIEW (CPL), B...",315.609891,4294.425418,fu qingru pai liu,"[fu, qingru, pai, liu]","[fu, qingru, pai, liu]","[fu, qingru, pai, liu]",Ranch


### First Middle & Last Name 

In [24]:
splitted = df1['Pre-Process'].str.split()
df1['first_name'] = splitted.str[0]
df1['last_name'] = splitted.str[-1]
df1['middle_name'] = splitted.str[1]
df1['middle_name'] = df1['middle_name'].mask(df1['middle_name'] == df1['last_name'], '')
df1.head()

C:\Users\Abu Huraira\AppData\Local\Temp\ipykernel_6456\2729165251.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['first_name'] = splitted.str[0]
C:\Users\Abu Huraira\AppData\Local\Temp\ipykernel_6456\2729165251.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['last_name'] = splitted.str[-1]
C:\Users\Abu Huraira\AppData\Local\Temp\ipykernel_6456\2729165251.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

,OID_,OWNER,MAIL_NAME,MAIL_ADDRE,MAIL_AD_01,MAIL_AD_02,M_RECPNT,M_USPSBOX,M_ADDRESSN,M_STREETPR,...,Shape_Length,Shape_Area,Pre-Process,Tokenized,SW_Rem,Lemmas,OwnerType,first_name,last_name,middle_name
0,1,AGUILLON HECTOR ETUX NEYDA TOBAR,,409 W AVENUE A,,BELTON TX 76513,,,409,W AVENUE,...,567.120628,15652.88791,aguillon hector etux neyda tobar,"[aguillon, hector, etux, neyda, tobar]","[aguillon, hector, etux, neyda, tobar]","[aguillon, hector, etux, neyda, tobar]",Person,aguillon,tobar,hector
1,2,CITY OF BELTON,,PO BOX 120,,BELTON TX 76513,,PO BOX 120,,,...,1364.987519,58986.80674,city of belton,"[city, of, belton]","[city, belton]","[city, belton]",NonProfit,city,belton,of
2,3,BELTON TEXAS REAL ESTATE HOLDINGS LLC,,600 SAGE BRUSH,,BELTON TX 76513,,,600,,...,411.048085,10105.52538,belton texas real estate holdings llc,"[belton, texas, real, estate, holdings, llc]","[belton, texas, real, estate, holdings, llc]","[belton, texas, real, estate, holding, llc]",Company,belton,llc,texas
3,4,LEWIS NELDA MAE,,PO BOX 1082,,BELTON TX 76513,,PO BOX 1082,,,...,989.135441,44147.27746,lewis nelda mae,"[lewis, nelda, mae]","[lewis, nelda, mae]","[lewis, nelda, mae]",Person,lewis,mae,nelda
4,5,MARQUEZ FERNANDO,,PO BOX 983,,BELTON TX 76513,,PO BOX 983,,,...,838.542479,43946.81203,marquez fernando,"[marquez, fernando]","[marquez, fernando]","[marquez, fernando]",Person,marquez,fernando,


In [25]:
df1['LEGAL_DES_Standardized']=df1['LEGAL_DESC'].str.split(",").str[0]

C:\Users\Abu Huraira\AppData\Local\Temp\ipykernel_6456\739862501.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['LEGAL_DES_Standardized']=df1['LEGAL_DESC'].str.split(",").str[0]


In [26]:
df1

,OID_,OWNER,MAIL_NAME,MAIL_ADDRE,MAIL_AD_01,MAIL_AD_02,M_RECPNT,M_USPSBOX,M_ADDRESSN,M_STREETPR,...,Shape_Area,Pre-Process,Tokenized,SW_Rem,Lemmas,OwnerType,first_name,last_name,middle_name,LEGAL_DES_Standardized
0,1,AGUILLON HECTOR ETUX NEYDA TOBAR,,409 W AVENUE A,,BELTON TX 76513,,,409,W AVENUE,...,15652.887910,aguillon hector etux neyda tobar,"[aguillon, hector, etux, neyda, tobar]","[aguillon, hector, etux, neyda, tobar]","[aguillon, hector, etux, neyda, tobar]",Person,aguillon,tobar,hector,BELTON ORIGINAL
1,2,CITY OF BELTON,,PO BOX 120,,BELTON TX 76513,,PO BOX 120,,,...,58986.806740,city of belton,"[city, of, belton]","[city, belton]","[city, belton]",NonProfit,city,belton,of,BELTON ORIGINAL
2,3,BELTON TEXAS REAL ESTATE HOLDINGS LLC,,600 SAGE BRUSH,,BELTON TX 76513,,,600,,...,10105.525380,belton texas real estate holdings llc,"[belton, texas, real, estate, holdings, llc]","[belton, texas, real, estate, holdings, llc]","[belton, texas, real, estate, holding, llc]",Company,belton,llc,texas,PENDLETON
3,4,LEWIS NELDA MAE,,PO BOX 1082,,BELTON TX 76513,,PO BOX 1082,,,...,44147.277460,lewis nelda mae,"[lewis, nelda, mae]","[lewis, nelda, mae]","[lewis, nelda, mae]",Person,lewis,mae,nelda,BELTON ORIGINAL
4,5,MARQUEZ FERNANDO,,PO BOX 983,,BELTON TX 76513,,PO BOX 983,,,...,43946.812030,marquez fernando,"[marquez, fernando]","[marquez, fernando]","[marquez, fernando]",Person,marquez,fernando,,RICHARD 2ND
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,1048571,WHITE BARBARA G,,9813 CLOCKTOWER CT,,PLANO TX 75025-6584,,,9813,,...,5037.827226,white barbara g,"[white, barbara, g]","[white, barbara, g]","[white, barbara, g]",Ranch,white,g,barbara,PASQUINELLIS CASTLEBROOK AT RIDGEVIEW (CPL)
1048571,1048572,BAUM RAQUEL,,9816 CLOCKTOWER CT,,PLANO TX 75025-6583,,,9816,,...,5123.142305,baum raquel,"[baum, raquel]","[baum, raquel]","[baum, raquel]",Ranch,baum,raquel,,PASQUINELLIS CASTLEBROOK AT RIDGEVIEW (CPL)
1048572,1048573,AZIZ HOOR,,9817 CLOCKTOWER CT,,PLANO TX 75025-6584,,,9817,,...,4294.791267,aziz hoor,"[aziz, hoor]","[aziz, hoor]","[aziz, hoor]",Ranch,aziz,hoor,,PASQUINELLIS CASTLEBROOK AT RIDGEVIEW (CPL)
1048573,1048574,FU QINGRU & PAI LIU &,,6409 LAMBEAU CT,,MCKINNEY TX 75070-9560,,,6409,,...,4294.425418,fu qingru pai liu,"[fu, qingru, pai, liu]","[fu, qingru, pai, liu]","[fu, qingru, pai, liu]",Ranch,fu,liu,qingru,PASQUINELLIS CASTLEBROOK AT RIDGEVIEW (CPL)


In [27]:
df1['Address'] = df1['MAIL_ADDRE'] +' '+ df1['MAIL_AD_02']
df1.head()

C:\Users\Abu Huraira\AppData\Local\Temp\ipykernel_6456\2851000327.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Address'] = df1['MAIL_ADDRE'] +' '+ df1['MAIL_AD_02']


,OID_,OWNER,MAIL_NAME,MAIL_ADDRE,MAIL_AD_01,MAIL_AD_02,M_RECPNT,M_USPSBOX,M_ADDRESSN,M_STREETPR,...,Pre-Process,Tokenized,SW_Rem,Lemmas,OwnerType,first_name,last_name,middle_name,LEGAL_DES_Standardized,Address
0,1,AGUILLON HECTOR ETUX NEYDA TOBAR,,409 W AVENUE A,,BELTON TX 76513,,,409,W AVENUE,...,aguillon hector etux neyda tobar,"[aguillon, hector, etux, neyda, tobar]","[aguillon, hector, etux, neyda, tobar]","[aguillon, hector, etux, neyda, tobar]",Person,aguillon,tobar,hector,BELTON ORIGINAL,409 W AVENUE A BELTON TX 76513
1,2,CITY OF BELTON,,PO BOX 120,,BELTON TX 76513,,PO BOX 120,,,...,city of belton,"[city, of, belton]","[city, belton]","[city, belton]",NonProfit,city,belton,of,BELTON ORIGINAL,PO BOX 120 BELTON TX 76513
2,3,BELTON TEXAS REAL ESTATE HOLDINGS LLC,,600 SAGE BRUSH,,BELTON TX 76513,,,600,,...,belton texas real estate holdings llc,"[belton, texas, real, estate, holdings, llc]","[belton, texas, real, estate, holdings, llc]","[belton, texas, real, estate, holding, llc]",Company,belton,llc,texas,PENDLETON,600 SAGE BRUSH BELTON TX 76513
3,4,LEWIS NELDA MAE,,PO BOX 1082,,BELTON TX 76513,,PO BOX 1082,,,...,lewis nelda mae,"[lewis, nelda, mae]","[lewis, nelda, mae]","[lewis, nelda, mae]",Person,lewis,mae,nelda,BELTON ORIGINAL,PO BOX 1082 BELTON TX 76513
4,5,MARQUEZ FERNANDO,,PO BOX 983,,BELTON TX 76513,,PO BOX 983,,,...,marquez fernando,"[marquez, fernando]","[marquez, fernando]","[marquez, fernando]",Person,marquez,fernando,,RICHARD 2ND,PO BOX 983 BELTON TX 76513


In [28]:
df1=df1[['OWNER','OwnerType','first_name','last_name','middle_name','LEGAL_DES_Standardized','Address']]

In [29]:
df1

,OWNER,OwnerType,first_name,last_name,middle_name,LEGAL_DES_Standardized,Address
0,AGUILLON HECTOR ETUX NEYDA TOBAR,Person,aguillon,tobar,hector,BELTON ORIGINAL,409 W AVENUE A BELTON TX 76513
1,CITY OF BELTON,NonProfit,city,belton,of,BELTON ORIGINAL,PO BOX 120 BELTON TX 76513
2,BELTON TEXAS REAL ESTATE HOLDINGS LLC,Company,belton,llc,texas,PENDLETON,600 SAGE BRUSH BELTON TX 76513
3,LEWIS NELDA MAE,Person,lewis,mae,nelda,BELTON ORIGINAL,PO BOX 1082 BELTON TX 76513
4,MARQUEZ FERNANDO,Person,marquez,fernando,,RICHARD 2ND,PO BOX 983 BELTON TX 76513
...,...,...,...,...,...,...,...
1048570,WHITE BARBARA G,Ranch,white,g,barbara,PASQUINELLIS CASTLEBROOK AT RIDGEVIEW (CPL),9813 CLOCKTOWER CT PLANO TX 75025-6584
1048571,BAUM RAQUEL,Ranch,baum,raquel,,PASQUINELLIS CASTLEBROOK AT RIDGEVIEW (CPL),9816 CLOCKTOWER CT PLANO TX 75025-6583
1048572,AZIZ HOOR,Ranch,aziz,hoor,,PASQUINELLIS CASTLEBROOK AT RIDGEVIEW (CPL),9817 CLOCKTOWER CT PLANO TX 75025-6584
1048573,FU QINGRU & PAI LIU &,Ranch,fu,liu,qingru,PASQUINELLIS CASTLEBROOK AT RIDGEVIEW (CPL),6409 LAMBEAU CT MCKINNEY TX 75070-9560


In [30]:
#!pip install geopy

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="myApp")

for i in df1.index:
    try:
        #tries fetch address from geopy
        location = geolocator.geocode(df1['Address'][i])
        
        #append lat/long to column using dataframe location
        df1.loc[i,'location_address'] = location.address
    except:
        #catches exception for the case where no value is returned
        #appends null value to column
        df1.loc[i,'location_address'] = ""

#print first rows as sample
df1.head()

C:\Users\Abu Huraira\AppData\Local\Temp\ipykernel_6456\2330189389.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.loc[i,'location_address'] = location.address


In [30]:
df1.to_csv("Result.csv")